In [44]:
import pandas as pd
import urllib.request
import requests
import json

In [45]:
def distance(s_lat, s_long, d_lat, d_long):
    api_key = 'HUUA5tWavRI0fJmIS4P7NydMieu4uona'
    url ='http://www.mapquestapi.com/directions/v2/routematrix?'
    source = str(s_lat)+','+str(s_long)
    dest = str(d_lat)+','+str(d_long)
    js = {'locations' : [source, dest] }
    r = requests.post(url + 'key=' + api_key , json = js)
    x = r.json()
    duration = x['time'][1]
    duration = float(duration/60)
    return duration

In [46]:
d1 = pd.read_csv('lat-long.csv')
d2 = pd.read_csv('place.csv')
d3 = pd.read_csv('weights.csv')

In [51]:
d3 = d3.drop(['Unnamed: 0'], axis=1)
d3

,Source,Destination,Weight
0,RGIA,Gangaram,30.766667
1,RGIA,Kurmaguda,23.450000
2,RGIA,Rajendranagar Mandal,21.100000
3,RGIA,Kismatpur,21.000000
4,Gangaram,Kurmaguda,29.450000
5,Kurmaguda,Kothapet Village,10.016667
6,Kurmaguda,Sivarampalli,17.566667
7,Rajendranagar Mandal,Sivarampalli,5.483333
8,Sivarampalli,Kothapet Village,22.866667
9,Kothapet Village,L B Nagar,3.983333


In [52]:
coord = {}
for i in range(len(d2['Localities'])):
    coord[d2['Localities'][i]] = (float(d1['latitude'][i]), float(d1['longitude'][i]))

In [53]:
adj = {}

for i in range(len(d2['Localities'])):
    adj[d2['Localities'][i]] = []

In [54]:
for i in range(len(d3['Source'])):
    adj[d3['Source'][i]].append((d3['Destination'][i],d3['Weight'][i]))
    adj[d3['Destination'][i]].append((d3['Source'][i],d3['Weight'][i]))

In [55]:
start = input("Enter Source: ")
end = input("Enter Destination: ")

Enter Source: RGIA
Enter Destination: BITS Hyderabad


In [56]:
h = {}

for i in range(len(d2['Localities'])):
    h[d2['Localities'][i]] = distance(str(coord[d2['Localities'][i]][0]), str(coord[d2['Localities'][i]][1]), str(coord[end][0]), str(coord[end][1]))

In [106]:
o_list = [(start, h[start])]
c_list = []
val = 1000000

f_list = {}
for i in d2['Localities']:
    f_list[i] = 1000000
    
parent = {}

f_list[start] = h[start]

In [107]:
while(len(o_list) > 0 and val > o_list[0][1]):
    
    nxt = o_list[0]
    del(o_list[0])
    c_list.append(nxt[0])
    
    if nxt[0] == end:
        val = nxt[1]
    
    for i in adj[nxt[0]]:
        
        f = nxt[1] - h[nxt[0]] + i[1] + h[i[0]]
        
        if f < f_list[i[0]]:
            parent[i[0]] = nxt[0]
            t = (i[0],f_list[i[0]])
            if t in o_list:
                o_list.remove(t)
            elif t in c_list:
                c_list.remove(t)
            o_list.append((i[0],f))
            f_list[i[0]] = f    
               

In [108]:
path = []
loc = end

while loc != start:
    path.append(loc)
    loc = parent[loc]

path.append(start)
path.reverse()    

In [109]:
ans_coord = []

for i in path:
    ans_coord.append(coord[i])

In [110]:
print('Path is :')
print(path)
print("Coordinates are :")
print(ans_coord)

Path is :
['RGIA', 'Kurmaguda', 'Kothapet Village', 'L B Nagar', 'Nagole', 'Uppal', 'Moula Ali', 'Sainikpuri', 'BITS Hyderabad']
Coordinates are :
[(17.231163600000002, 78.43178336758146), (17.3607708, 78.5024039), (17.3740429, 78.54361259999999), (17.349846, 78.5479412), (17.3775306, 78.56012309999998), (17.402509100000003, 78.5612562), (17.466816100000003, 78.5592161), (17.4892154, 78.54248890000001), (17.54291415, 78.57149372294117)]


In [111]:
url = "https://dev.virtualearth.net/REST/v1/Imagery/Map/Road/Routes?"
for i in range(len(ans_coord)):
    url = url + "wp."+str(i)+"="+str(ans_coord[i][0])+","+str(ans_coord[i][1])+"&"
url = url + "key=Ar8arSG0MGv65wos9fz8B5kLH6N_iotpLGTre51uw7kce5q-D5myBJriG2CyBft1"

In [112]:
print(url)

https://dev.virtualearth.net/REST/v1/Imagery/Map/Road/Routes?wp.0=17.231163600000002,78.43178336758146&wp.1=17.3607708,78.5024039&wp.2=17.3740429,78.54361259999999&wp.3=17.349846,78.5479412&wp.4=17.3775306,78.56012309999998&wp.5=17.402509100000003,78.5612562&wp.6=17.466816100000003,78.5592161&wp.7=17.4892154,78.54248890000001&wp.8=17.54291415,78.57149372294117&key=Ar8arSG0MGv65wos9fz8B5kLH6N_iotpLGTre51uw7kce5q-D5myBJriG2CyBft1
